In [ ]:
def xyCoordinates(coordinates:list, x_shape:int, y_shape:int): 
    #Remember that the coordinates are written like x1;y1;x2;y2;x3;y3
    if type(coordinates)!=list:
        print(type(coordinates))
        raise ValueError
    xyCoords = [] #The format will be [[x01,y01,x02,y02,...],[x11,y11,x12,y12,.....],.........]
    
    
    for coords in coordinates:
        xys = [] #These are the [x01,y01,x02,y02,...] described above

        coordslist = coords.split(";") #Remember that every value indexed at a even num is xs, and odd num is ys
        for i in range(len(coordslist)):
            xys.append(int(coordslist[i]))
           
       
        #############################
        #For debugging
        if width == 0 and height == 0: 
            width, height = 2, 3 
        
        
        xyCoords.append(xys)
    
    return xyCoords

In [ ]:
#Read in the textfile with wher the nodule imformation is pou up like this: 
    # NoduleID: SOPInstanceUID: png path: coordinates: 
    # Ex:
    # Nodule 01,1.3.2.4.6.3.12413......,/home/heris/............../image1.png,x1;y1;x2;y2;x3;y3

def segmentYOLOformat(input_txt_path:str, output_txt_path:str):
    try:
        NoduleIDs = []
        CoordinatesTot = []
        with open(input_txt_path, "r") as txt:
            lines = txt.readlines()
            for line in lines:
                #Each line is in a csv format so can split on ","
                line_split = line.split(",")
                noduleID, SOPInstanceUID, img_path = line_split[0], line_split[1], line_split[2]
                NoduleIDs.append(noduleID)   
                x_shape, y_shape = int(line_split[3]), int(line_split[4])
                coordinates = line_split[5]
                CoordinatesTot.append(coordinates) 
        coordinates = xyCoordinates(CoordinatesTot,x_shape, y_shape)
        ##################################################################
        #if the lesion is a point lesion, the txt file is then put into tmp
        #Point lesions will be the top point of the tumor ad non-nodules.
        output_txt_copy_path = output_txt_path
        output_tmp_path = "/home/heris/data/pointlesions_trash/trash.txt" #tmp path
        output_txt_path = output_tmp_path
        for xys in coordinates: 
            if len(xys)!=2:
                output_txt_path = output_txt_copy_path #if the whole image contains at least one normal lesion, we will eliminate the point lesions later
                break 
        ##################################################################

    except FileNotFoundError:
        print(f"File {input_txt_path} not found.")
    except SyntaxError:
        print(f"Wrong imput syntax!")
    
    #JASON or TXT, read up on YOLO v8
    try:
        with open(output_txt_path, 'w') as file:
            # Write the content to the file
            #file.write("0 ") #Lesion ID = 0 
            #######
            #Modify the following line, take into accunt spherecy and malignincy to make other lesion labels etc lesion with malign < 3 & spherecy < 3 => Label 0, malign < 3 => Label 1, spherecy < 3 => Label 2, malign > 3 & spherecy > 3 => Label 3 etc. 
            ####### 
            
            for xys in coordinates:
                #file.write("0 ")
                for i in range(len(xys)):
                    if len(xys) != 2:
                        if i == 0:
                            file.write("0 ")
                        if i != len(xys)-1:
                            file.write(f"{xys[i]} ")
                        else:
                            file.write(f"{xys[i]}\n")
               
            
        print(f"Text has been written to {output_txt_path}.")
    except Exception as e:
        print(f"An error occurred: {e}")

    return SOPInstanceUID, img_path, NoduleIDs, coordinates, output_txt_path 

In [ ]:
import os
import shutil
import math 

In [ ]:
#Automated script 
output_folder_path = "/home/heris/data/RAW/YOLO_pre_dataset"
input_txt_path = "/home/heris/data/RAW/LIDC-IDRI-1000png"
if not os.path.exists(output_folder_path):
    os.mkdir(output_folder_path)

for root, dirs, files in os.walk(input_txt_path):
    if len(dirs) == 0:
        for file in files: 
            if file[-1] == "t":
                if not os.path.exists(output_folder_path+"/"+file):
                    if os.path.exists(root+"/"+file[0:-3]+"png"):
                        _, _, _ , _, txt_path = boundingBoxDim(root+"/"+file,output_folder_path+"/"+file)
                        if txt_path != "/home/heris/data/pointlesions_trash/trash.txt":
                            shutil.copy(root+"/"+file[0:-3]+"png",output_folder_path)